In [46]:
%matplotlib inline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import json
from pprint import pprint
from pandas.io.json import json_normalize

import re

def clean_address(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    cleantext = re.sub("\s\s+", "",cleantext)
    return cleantext.strip()

def clean_price(price):
    prices = re.findall(r'\d+', price)
    if not len(prices):
        return {}
    if len(prices) == 1:
        return {'min_price': prices[0]}
    
    min_price = min(prices)
    max_price = max(prices)
    return {'min_price': min_price.strip(), 'max_price': max_price.strip()}

def assert_flat(test_dict):
    for key in test_dict:
        assert type(test_dict[key]) == str

In [47]:
# setup
filename = '1225114.json'

# Load file

In [48]:
with open('../data/json/' + filename) as data_file:    
    data = json.load(data_file)
data.keys()

dict_keys(['Reviews', 'HotelInfo'])

# HotelInfo

In [49]:
hotel_info = data['HotelInfo']

if 'Address' in hotel_info:
    hotel_info['Address'] = clean_address(hotel_info['Address'])
hotel_info.update(clean_price(hotel_info['Price']))
del hotel_info['Price']
assert_flat(hotel_info)
hotel_info

{'HotelID': '1225114',
 'HotelURL': 'http://www.tripadvisor.com/ShowUserReviews-g60878-d1225114-r23287166-Fairmont_Olympic_Hotel-Seattle_Washington.html'}

# Reviews

In [52]:
reviews = data['Reviews']

for review in reviews:
    review['HotelID'] = hotel_info['HotelID']
    # remove the quotes:
    if 'Title' in review:
        review['Title'] = review['Title'].strip('“').strip('”')
    if 'Ratings' in review:
        for rating in review['Ratings']:
            review['rating' + rating] = review['Ratings'][rating]
        del review['Ratings']
    assert_flat(review)